In [1]:
import pandas as pd
import numpy as np
import obspy
import matplotlib.pyplot as plt
from geopy.distance import geodesic
import math

In [2]:
# Load the data from the file
file_path = 'reloc.txt'
df = pd.read_csv(file_path, header=None, delimiter='\s+')

#Rename the columns as specified
column_names = ["ID", "LAT", "LON", "DEPTH", "X", "Y", "Z", "EX", "EY", "EZ", 
                "YR", "MO", "DY", "HR", "MI", "SC", "MAG", "NCCP", "NCCS", 
                "NCTP", "NCTS", "RCC", "RCT", "CID"]
df.columns = column_names


df.head()

,ID,LAT,LON,DEPTH,X,Y,Z,EX,EY,EZ,...,MI,SC,MAG,NCCP,NCCS,NCTP,NCTS,RCC,RCT,CID
0,2,35.711609,-117.491626,13.970,3688.3,4550.2,5229.1,212.3,185.2,247.0,...,2,55.08,4.3,0,0,278,290,-9.0,0.102,1
1,3,35.706991,-117.494873,13.379,3394.6,4038.1,4637.9,265.6,216.7,269.7,...,4,2.36,2.0,0,0,123,125,-9.0,0.183,1
2,5,35.706718,-117.501099,8.922,2831.1,4007.8,181.6,310.1,226.5,293.1,...,5,1.68,1.0,0,0,54,20,-9.0,0.173,1
3,6,35.711161,-117.489071,13.184,3919.6,4500.9,4443.1,254.7,217.4,271.7,...,5,5.16,1.3,0,0,104,152,-9.0,0.156,1
4,7,35.710189,-117.487158,13.380,4092.8,4392.8,4639.2,503.6,372.0,294.9,...,5,47.88,0.6,0,0,95,85,-9.0,0.163,1


# New Dataframe RELOC

In [26]:
# Separate the seconds into whole seconds and fractional seconds
df['whole_seconds'] = df['SC'].apply(lambda x: str(int(float(x))))
df['fractional_seconds'] = df['SC'].apply(lambda x: f".{str(float(x)).split('.')[1]}" if '.' in str(x) else '.0')

# Combine the date and time components
df['origin_time'] = pd.to_datetime(df['YR'].astype(str) + '-' + 
                                   df['MO'].astype(str).str.zfill(2) + '-' + 
                                   df['DY'].astype(str).str.zfill(2) + ' ' + 
                                   df['HR'].astype(str).str.zfill(2) + ':' + 
                                   df['MI'].astype(str).str.zfill(2) + ':' + 
                                   df['whole_seconds'], 
                                   errors='coerce', format='%Y-%m-%d %H:%M:%S')

# Add the fractional seconds
df['origin_time'] = df['origin_time'] + pd.to_timedelta(df['fractional_seconds'].astype(float), unit='s')

# Update the reloc_df DataFrame
reloc_df = df[['ID', 'origin_time', 'LON', 'LAT', 'DEPTH', 'MAG', 'RCT']].copy()
reloc_df.rename(columns={'ID':'id', 'LON': 'lon', 'LAT': 'lat', 'DEPTH': 'depth', 'MAG': 'magnitude', 'RCT': 'rct'}, inplace=True)

reloc_df.head()

,id,origin_time,lon,lat,depth,magnitude,rct
0,2,2019-07-04 17:02:55.080,-117.491626,35.711609,13.970,4.3,0.102
1,3,2019-07-04 17:04:02.360,-117.494873,35.706991,13.379,2.0,0.183
2,5,2019-07-04 17:05:01.680,-117.501099,35.706718,8.922,1.0,0.173
3,6,2019-07-04 17:05:05.160,-117.489071,35.711161,13.184,1.3,0.156
4,7,2019-07-04 17:05:47.880,-117.487158,35.710189,13.380,0.6,0.163


In [28]:
reloc_df.to_csv("reloc.csv",index=False)